In [2]:
from __future__ import print_function

import sys
import re
from operator import add 
import pandas as pd
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import json
import boto
import boto3
from boto.s3.key import Key
import boto.s3.connection
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().setAppName("first")
#sc = SparkContext(conf=conf)
spark._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId",'xxx')
spark._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey",'xxx')
config_dict = {"fs.s3n.awsAccessKeyId":"xxx",
               "fs.s3n.awsSecretAccessKey":"xxx"}
bucket = "project4capstones3"
prefix = "/2017/06/16//14/project*"
filename = "s3n://{}/{}".format(bucket, prefix)
warehouse_location = 'spark-warehouse'
rdd = sc.hadoopFile(filename,
            'org.apache.hadoop.mapred.TextInputFormat',
            'org.apache.hadoop.io.Text',
            'org.apache.hadoop.io.LongWritable',
            conf=config_dict)
spark_sess = SparkSession \
        .builder \
        .appName("Python Spark SQL Hive integration example") \
        .config("spark.sql.warehouse.dir", warehouse_location) \
        .enableHiveSupport() \
        .getOrCreate()
# spark_sess = SparkSession.builder.appName("PythonWordCount").getOrCreate()
#spark_sess.read.json(rdd).show()
#spark_sess.read.json(rdd.map(lambda x: x[1])).show()

In [41]:
df = spark_sess.read.json(rdd.map(lambda x: x[1]))

In [42]:
data_rm_na = df.filter(df['status_id']!='None')

In [43]:
features_of_interest = ['user_followers_count', 'user_friends_count', 'user_statuses_count', 'rt_status_retweet_count', 'rt_status_favorite_count',
                        'rt_status_entities_user_mentions','user_listed_count', 'status_num_mentions', 'status_retweet_count', 'status_geo',
                        'status_hashtags','user_name', 'user_loc', 'status_text', 'user_desc', 'user_img_url', 'user_id', 'searched_names', 
                        'status_sentMag', 'status_sentScore']

In [44]:
df_reduce= data_rm_na.select(features_of_interest)
df_reduce = df_reduce.withColumn("user_followers_count", df_reduce["user_followers_count"].cast(IntegerType()))
df_reduce = df_reduce.withColumn("user_friends_count", df_reduce["user_friends_count"].cast(IntegerType()))
df_reduce = df_reduce.withColumn("user_statuses_count", df_reduce["user_statuses_count"].cast(IntegerType()))
df_reduce = df_reduce.withColumn("rt_status_retweet_count", df_reduce["rt_status_retweet_count"].cast(IntegerType()))
df_reduce = df_reduce.withColumn("user_listed_count", df_reduce["user_listed_count"].cast(IntegerType()))
df_reduce = df_reduce.withColumn("status_num_mentions", df_reduce["status_num_mentions"].cast(IntegerType()))
df_reduce = df_reduce.withColumn("status_retweet_count", df_reduce["status_retweet_count"].cast(IntegerType()))
# df_reduce = df_reduce.withColumn(“status_sentMag”, df_reduce[“status_sentMag”].cast(DoubleType()))
# df_reduce = df_reduce.withColumn(“status_sentScore”, df_reduce[“status_sentScore”].cast(DoubleType()))
df_reduce = df_reduce.withColumn("rt_status_favorite_count", df_reduce["rt_status_favorite_count"].cast(IntegerType()))

In [45]:
# for i in numeric_features:
#     print(i)
#     df_reduced_num = df_reduced_num.withColumn(i, df_reduced_num[i].cast('float'))
# #     df_reduced_num.select(df_reduced_num.user_followers_count.cast('float'))

In [46]:
max_user_followers_count = df_reduce.select('user_followers_count').rdd.flatMap(list).max()


df_reduce = df_reduce.withColumn('user_followers_count', df_reduce['user_followers_count'] / max_user_followers_count)
### reputation ####
df_reduce = df_reduce.withColumn('reputation',(df_reduce.user_followers_count*df_reduce.user_statuses_count)/(df_reduce.user_friends_count+1)+10)
#listed_count rescale
max_listed_count = df_reduce.select('user_listed_count').rdd.flatMap(list).max()
df_reduce = df_reduce.withColumn('user_listed_count',df_reduce.user_listed_count/max_listed_count)

### RESONANCE #####
#status_num_count rescale 
max_status_num_mentions  = df_reduce.select('status_num_mentions').rdd.flatMap(list).max()
df_reduce = df_reduce.withColumn('status_num_mentions',df_reduce.status_num_mentions/max_status_num_mentions)

# retweets_received rescale
max_retweets_received  = df_reduce.select('rt_status_retweet_count').rdd.flatMap(list).max()
df_reduce = df_reduce.withColumn('retweets_received',df_reduce.rt_status_retweet_count/max_retweets_received)

### ENGAGEMENT ###
# mentions_ratio
df_reduce= df_reduce.withColumn('mentions_ratio',df_reduce.status_num_mentions/(df_reduce.user_statuses_count+1)+1)

# retweets_given_ratio
df_reduce= df_reduce.withColumn('retweets_given_ratio',df_reduce.status_retweet_count/(df_reduce.user_statuses_count+1)+1)

#number_of_tweets rescale
max_user_statuses_count  = df_reduce.select('user_statuses_count').rdd.flatMap(list).max()
df_reduce = df_reduce.withColumn('number_of_tweets',df_reduce.user_statuses_count/max_user_statuses_count)

root
 |-- user_followers_count: double (nullable = true)
 |-- user_friends_count: integer (nullable = true)
 |-- user_statuses_count: integer (nullable = true)
 |-- rt_status_retweet_count: integer (nullable = true)
 |-- rt_status_favorite_count: integer (nullable = true)
 |-- rt_status_entities_user_mentions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- user_listed_count: double (nullable = true)
 |-- status_num_mentions: double (nullable = true)
 |-- status_retweet_count: integer (nullable = true)
 |-- status_geo: string (nullable = true)
 |-- status_hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- user_name: string (nullable = true)
 |-- user_loc: string (nullable = true)
 |-- status_text: string (nullable = true)
 |-- user_desc: string (nullable = true)
 |-- user_img_url: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- searched_names: string (nullable = true)
 |-- status_sentMag: double (null

In [28]:
type(df_reduce)

pyspark.sql.dataframe.DataFrame

In [14]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [32]:
df_reduce.createOrReplaceTempView("twitterHive")

In [33]:
spark.sql("SHOW TABLES").show()

+--------+-----------+-----------+
|database|  tableName|isTemporary|
+--------+-----------+-----------+
|        |twitterhive|       true|
+--------+-----------+-----------+



In [34]:
spark.sql("SELECT * FROM twitterHive").show()

+--------------------+------------------+-------------------+-----------------------+------------------------+--------------------------------+--------------------+-------------------+--------------------+----------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------+--------------+----------------+------------------+--------------------+------------------+--------------------+--------------------+
|user_followers_count|user_friends_count|user_statuses_count|rt_status_retweet_count|rt_status_favorite_count|rt_status_entities_user_mentions|   user_listed_count|status_num_mentions|status_retweet_count|status_geo|     status_hashtags|         user_name|            user_loc|         status_text|           user_desc|        user_img_url|           user_id|searched_names|status_sentMag|status_sentScore|        reputation|   retweets_received|    mentions_ratio|retweets_given_ratio|    nu

In [36]:
df_writer = pyspark.sql.DataFrameWriter(df_reduce)

In [38]:
df_writer.saveAsTable('test_table', mode='overwrite')

Py4JJavaError: An error occurred while calling o833.saveAsTable.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply$mcV$sp(FileFormatWriter.scala:147)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:57)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:101)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	at or